In [1]:
from imblearn.over_sampling import SMOTE
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
import numpy as np
import cv2
import os


In [2]:
# Load dataset
def load_images_and_labels(dataset_path, img_size=(224, 224)):
    images, labels = [], []
    for label_dir in os.listdir(dataset_path):
        for img_file in os.listdir(os.path.join(dataset_path, label_dir)):
            img_path = os.path.join(dataset_path, label_dir, img_file)
            image = cv2.imread(img_path)
            image = cv2.resize(image, img_size)
            images.append(image)
            labels.append(label_dir)
    return np.array(images), np.array(labels)

images, labels = load_images_and_labels("./wheat_leaf")
images = images / 255.0  # Normalize the images to [0, 1] range

In [3]:
# Encode labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)


In [4]:
# Split into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(images, labels, test_size=0.3, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


In [5]:
# Check class distribution in the training set
from collections import Counter
class_distribution = Counter(np.argmax(y_train, axis=1))
print("Class distribution in the training set:", class_distribution)

# Apply SMOTE for oversampling the minority class
X_train_flattened = X_train.reshape(X_train.shape[0], -1)  # Flatten images for SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train_flattened, y_train)

# Reshape back to image dimensions after SMOTE
X_resampled = X_resampled.reshape(-1, 224, 224, 3)


Class distribution in the training set: Counter({2: 150, 1: 70, 0: 64})


In [6]:
# Build the model using MobileNetV2
mobilenet = MobileNetV2(input_shape=(224, 224, 3), weights="imagenet", include_top=False)
mobilenet.trainable = False  # Freeze the MobileNetV2 layers


In [7]:
model = models.Sequential([
    mobilenet,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),  # Dense layer for feature learning
    layers.Dense(len(lb.classes_), activation='softmax')  # Output layer for classification
])


In [8]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])


In [9]:
# Train the model
model.fit(X_resampled, y_resampled, epochs=20, validation_data=(X_valid, y_valid))

Epoch 1/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 30s 896ms/step - accuracy: 0.6560 - loss: 0.6962 - val_accuracy: 0.8689 - val_loss: 0.2972
Epoch 2/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 8s 536ms/step - accuracy: 0.9673 - loss: 0.1179 - val_accuracy: 0.9180 - val_loss: 0.2110
Epoch 3/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 8s 504ms/step - accuracy: 0.9866 - loss: 0.0543 - val_accuracy: 0.9180 - val_loss: 0.1905
Epoch 4/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 8s 511ms/step - accuracy: 0.9984 - loss: 0.0301 - val_accuracy: 0.9016 - val_loss: 0.2096
Epoch 5/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 8s 558ms/step - accuracy: 1.0000 - loss: 0.0185 - val_accuracy: 0.9180 - val_loss: 0.1934
Epoch 6/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 8s 522ms/step - accuracy: 1.0000 - loss: 0.0116 - val_accuracy: 0.9672 - val_loss: 0.1497
Epoch 7/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 8s 513ms/step - accuracy: 1.0000 - loss: 0.0142 - val_accuracy: 0.9180 - val_loss: 0.2006
Epoch 8/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 8s 507ms/step - accuracy: 1.0000 - loss: 0.0059 - val_accuracy: 0

In [10]:

# Evaluate on the test set
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Test accuracy:", test_acc)


2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - accuracy: 0.9150 - loss: 0.1607
Test accuracy: 0.9193548560142517
